<a href="https://colab.research.google.com/github/ReaganSanz/Netflix-Recommender-LLM-Fine-Tuning-/blob/main/Fine_Tuning_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Fine-Tuning Mistral on Google Colab for Movie Recommendations (Netflix Dataset)

# Install dependencies
!pip install -q transformers datasets peft accelerate bitsandbytes


In [ ]:
# Step 1: Upload and prepare the dataset
# from google.colab import files
# import pandas as pd
# from tqdm import tqdm

# uploaded = files.upload()
# df = pd.read_csv(list(uploaded.keys())[0])
# df = df[df['type'] == 'Movie']
# df = df.dropna(subset=['description'])

# with open("train.txt", "w") as f:
#     for _, row in tqdm(df.iterrows(), total=len(df)):
#         prompt = f"### Instruction:\nRecommend a Netflix movie based on this input.\n\n### Input:\nGenre: {row['listed_in']}\nYear: {row['release_year']}\n\n### Response:\nYou might enjoy '{row['title']}' — {row['description'].strip()}"
#         f.write(prompt + "\n\n")

import os
import pandas as pd
from tqdm import tqdm

# Path to the Datasets folder
datasets_path = os.path.join(os.path.dirname(__file__), "Datasets")

# Combine all relevant CSV files
all_data = []
for file_name in os.listdir(datasets_path):
    if file_name.endswith(".csv"):
        file_path = os.path.join(datasets_path, file_name)
        df = pd.read_csv(file_path)
        if 'type' in df.columns and 'description' in df.columns:
            df = df[df['type'] == 'Movie']
            df = df.dropna(subset=['description'])
            all_data.append(df)

# Combine all data into a single DataFrame
combined_df = pd.concat(all_data, ignore_index=True)

# Save the combined data to a training file
with open("train.txt", "w") as f:
    for _, row in tqdm(combined_df.iterrows(), total=len(combined_df)):
        prompt = f"### Instruction:\nRecommend a Netflix movie based on this input.\n\n### Input:\nGenre: {row['listed_in']}\nYear: {row['release_year']}\n\n### Response:\nYou might enjoy '{row['title']}' — {row['description'].strip()}"
        f.write(prompt + "\n\n")


Saving netflix_titles.csv to netflix_titles.csv


100%|██████████| 6131/6131 [00:00<00:00, 9949.34it/s]


In [ ]:
# Step 2: Load Mistral model and tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Step 3: Load and tokenize dataset manually
with open("train.txt", "r") as f:
    data = f.read().split("\n\n")

train_inputs = tokenizer(data, return_tensors="pt", padding=True, truncation=True, max_length=512)
input_ids = train_inputs["input_ids"].to(model.device)
attention_mask = train_inputs["attention_mask"].to(model.device)



In [ ]:
# Step 4: Manually train the model
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)

for epoch in range(1):
    for i in range(0, len(input_ids), 1):
        input_batch = input_ids[i:i+1]
        mask_batch = attention_mask[i:i+1]

        outputs = model(input_batch, attention_mask=mask_batch, labels=input_batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 100 == 0:
            print(f"Epoch {epoch+1}, Step {i}, Loss: {loss.item()}")


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch 1, Step 0, Loss: 8.769152641296387


In [ ]:
# Step 5: Save model
model.save_pretrained("movie-gpt2")
tokenizer.save_pretrained("movie-gpt2")

NameError: name 'model' is not defined

In [ ]:
# Step 6: Test the fine-tuned model
prompts = [
    "### Instruction:\nRecommend a Netflix movie based on this input.\n\n### Input:\nGenre: Horror\nYear: 2019\n\n### Response:",
    "### Instruction:\nRecommend a Netflix movie based on this input.\n\n### Input:\nGenre: Comedy\nYear: 2021\n\n### Response:",
    "### Instruction:\nRecommend a Netflix movie based on this input.\n\n### Input:\nGenre: Drama, Romance\nYear: 2020\n\n### Response:"
]

model.eval()
for p in prompts:
    input_ids = tokenizer(p, return_tensors="pt").input_ids.to(model.device)
    with torch.no_grad():
        output = model.generate(input_ids, max_new_tokens=100)
    print(tokenizer.decode(output[0], skip_special_tokens=True))
    print("\n" + "="*80 + "\n")

